In [1]:
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('latin')
corpus_importer.import_corpus('latin_models_cltk')

In [2]:
from cltk.tag.pos import POSTag
tagger = POSTag('latin')

In [73]:
import glob
paths = glob.glob('./texts/perseus/*.txt')
#print(len(paths))
word_list = []
sentences_list = []
for p in paths:
    #sample_text_path = "./texts/perseus/phi0448.phi001.perseus-lat1.tb.txt"
    file = open(p)
    string = file.read()
    text = string
    file.close()
    for i in range(10):
        s = '[' + str(i) + ']/' 
        text = text.replace(s,'')
    text = text.replace('?', '.')
    text = text.replace('!', '.')
    text = text.replace('\n','')
    text = text.replace('[0]/','')
    text = text.replace('[1]/','')
    sentence_list = text.split('.')
    strip = {'',',',';'}
    for sent in sentence_list:
        words = sent.split(' ')
        tuple_list = []
        for word in words:
            word_tag_pair = word.split('/')
            if (len(word_tag_pair) == 2) and (word_tag_pair[0] not in strip):
                tuple_list.append( (word_tag_pair[0], word_tag_pair[1]) )
        if len(tuple_list) > 0:
            word_list.append(tuple_list)

In [57]:
print(sentence_list)

['Initium/n-s---nn- mihi/p-s---md- operis/n-s---ng- Servius/n-s---mn- Galba/n-s---mn- iterum/d-------- Titus/n-s---mn- Vinius/n-s---mn- consules/n-p---mn- erunt/v3pfia--- ', '/u-------- ,/u-------- \nnam/c-------- post/r-------- conditam/v-srppfa- urbem/n-s---fa- octingentos/m-p---ma- et/c-------- viginti/m-------- prioris/a-s---ngc aevi/n-s---ng- annos/n-p---ma- multi/a-p---mn- auctores/n-p---mn- rettulerunt/v3pria--- ,/u-------- dum/c-------- res/n-p---fn- populi/n-s---mg- Romani/a-s---mg_ memorabantur/v3piip--- pari/a-s---fb_ eloquentia/n-s---fb- ac/c-------- libertate/n-s---fb- :/u-------- \npostquam/c-------- bellatum/v-srppnn- apud/r-------- Actium/n-s---na- atque/c-------- omnem/a-s---fa- potentiam/n-s---fa- ad/r-------- unum/m-s---ma- conferri/v--pnp--- pacis/n-s---fg- interfuit/v3sria--- ,/u-------- magna/a-p---nn- illa/p-p---nn- ingenia/n-p---nn- cessere/v3pria--- ;/u-------- \nsimul/d-------- veritas/n-s---fn- pluribus/a-p---mbc modis/n-p---mb- infracta/v-srppfn- ,/u--------

In [75]:
#create dictionaries to store result
token_correct = {
    "TnT": 0,
    "CRF": 0,
    "Back-off": 0
}
num_sentence_correct = {
    "TnT": 0,
    "CRF": 0,
    "Back-off": 0
}

#counter for total number of words and sentences
total_sent = 0
total_word = 0

for sent in word_list[:500]:
    total_sent += 1
    sentence_correct = {
        "TnT": True,
        "CRF": True,
        "Back-off": True
    }
    #putting the sentence back together
    sentence_text = ""
    for word in sent:
        sentence_text = sentence_text + word[0]
        sentence_text += ' '
    
    #tag the sentences
    tnttags = tagger.tag_tnt(sentence_text)
    backofftags = tagger.tag_ngram_123_backoff(sentence_text)
    crftags = tagger.tag_crf(sentence_text)
    
    #compare to out tags
    for i in range(len(sent)):
        total_word += 1
        
        if sent[i][1] == tnttags[i][1].lower():
            token_correct['TnT'] += 1
        else:
#             if sent[i][0] != tnttags[i][0]:
#                 print(sent[i][0] + " " + tnttags[i][0])
            sentence_correct['TnT'] = False
        if backofftags[i][1] is not None and sent[i][1] == backofftags[i][1].lower():
            token_correct['Back-off'] += 1
        else:
            sentence_correct['Back-off'] = False
        if sent[i][1] == crftags[i][1].lower():
            token_correct['CRF'] += 1
        else:
            sentence_correct['CRF'] = False
            
    if sentence_correct['TnT']:
        num_sentence_correct['TnT'] += 1
    if sentence_correct['Back-off']:
        num_sentence_correct['Back-off'] += 1
    if sentence_correct['CRF']:
        num_sentence_correct['CRF'] += 1   

In [78]:
print(token_correct)

{'TnT': 7805, 'CRF': 6743, 'Back-off': 7564}


In [79]:
#print out results
tagger_list = ['TnT', 'CRF', 'Back-off']
for i in tagger_list:
    print("Tagger: " + i)
    token_accuracy = token_correct[i]/total_word
    print("Token Accuracy: " + str(token_accuracy))
    sentence_accuracy = num_sentence_correct[i]/total_sent
    print("Sentence: Accuracy: " + str(sentence_accuracy) + "\n")

Tagger: TnT
Token Accuracy: 0.8485540334855404
Sentence: Accuracy: 0.118

Tagger: CRF
Token Accuracy: 0.7330941509023701
Sentence: Accuracy: 0.028

Tagger: Back-off
Token Accuracy: 0.822352685366384
Sentence: Accuracy: 0.092



In [83]:
print(len(word_list))
print(word_list[0])

3842
[('Omnis', 'a-p---ma-'), ('homines', 'n-p---ma-'), ('qui', 'p-p---mn-'), ('sese', 'p-p---ma-'), ('student', 'v3ppia---'), ('praestare', 'v--pna---'), ('ceteris', 'a-p---nd-'), ('animalibus', 'n-p---nd-'), ('summa', 'a-s---fb-'), ('ope', 'n-s---fb-'), ('niti', 'v--pnp---'), ('decet', 'v3spia---'), ('ne', 'c--------'), ('vitam', 'n-s---fa-'), ('silentio', 'n-s---nb-'), ('transeant', 'v3ppsa---'), ('veluti', 'd--------'), ('pecora', 'n-p---nn-'), ('quae', 'p-p---na-'), ('natura', 'n-s---fn-'), ('prona', 'a-p---na-'), ('atque', 'c--------'), ('ventri', 'n-s---md-'), ('oboedientia', 'v-pppana-'), ('finxit', 'v3sria---')]
